# Plot

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os
import csv

In [2]:
replicate_result_folder = 'replicate_results'
if not os.path.exists(replicate_result_folder):
    os.makedirs(replicate_result_folder)

In [3]:
d = 5

# color setting
truth_color = "#FF6B6B"
est_color = "#4D96FF"
refined_color = "#6BCB77"
upper_labels=["\\theta_1","\\theta_2","\\theta_3","\\theta_4","\\theta_5"]

In [5]:
nn_ps_samples_path = os.path.join(os.getcwd(), 'result','nn_ps')
nn_mcmc_samples_path = os.path.join(os.getcwd(), 'result','nn_mcmc_samples')

bf_ps_samples_path = os.path.join(os.getcwd(), 'result','bf_ps')
bf_mcmc_samples_path = os.path.join(os.getcwd(), 'result','bf_mcmc_samples')

dnnabc_ps_samples_path = os.path.join(os.getcwd(), 'result','dnnabc_ps')

w2abc_ps_samples_path = os.path.join(os.getcwd(), 'result','w2abc_ps')

In [6]:
true_posterior_path = os.path.join(os.getcwd(), 'data','ps.npy')
true_posterior_samples = np.load(true_posterior_path)
print(true_posterior_samples.shape)
bandwidth_path = os.path.join(os.getcwd(), 'data','h_mmd.npy')
bandwidth = np.load(bandwidth_path)

(10000, 5)


In [7]:
def plot(mcmc_samples,ps_samples,true_posterior_samples, address, model="NN"):
    sns.set_style("whitegrid")
    fig, axs = plt.subplots(1, 5, figsize=(25, 6))
    true_ps = [1, 1, -1.0, -0.9, 0.6]

    # 定义每个theta_i对应的x轴范围
    x_limits = [
        [0.7, 1.3],  # theta_0
        [0.6, 1.4],  # theta_1
        [-1.5, 1.5],  # theta_2
        [-1.5, 1.5],  # theta_3
        [0, 1.2],  # theta_4
    ]

    for j, ax in enumerate(axs):
        ax.set_xlim(x_limits[j])
        ax.set_xticks(np.linspace(x_limits[j][0], x_limits[j][1], 5))

    for upper_label, j in zip(upper_labels,range(d)):
        sns.kdeplot(
            true_posterior_samples[:, j],
            ax=axs[j],
            fill=False,
            label="posterior",
            color=truth_color,
            linestyle="-.",
            linewidth=1.5,
        )
        sns.kdeplot(
            ps_samples[:, j],
            ax=axs[j],
            fill=False,
            label=f"{model}",
            color=est_color,
            linestyle="-",
            linewidth=1.5,
        )
        sns.kdeplot(
            mcmc_samples[:, j],
            ax=axs[j],
            fill=False,
            label=f"{model}+ABC-MCMC",
            color=refined_color,
            linestyle="--",
            linewidth=1.5,
        )
        axs[j].set_title(f"${upper_label}$", pad=15)
        axs[j].set_ylabel("")

    # save figure
    handles, labels = axs[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc="lower center", ncol=3)
    plt.tight_layout(pad=3.0)
    plt.savefig(address, dpi=300)
    plt.close(fig)

In [8]:
def subplot(mcmc_samples,ps_samples,true_posterior_samples, address, model="NN", element_index=0):
    sns.set_style("whitegrid")
    fig, axs = plt.subplots(1, 1, figsize=(6, 6))
    true_ps = [1, 1, -1.0, -0.9, 0.6]

    # 定义每个theta_i对应的x轴范围
    x_limits = [
        [0.7, 1.3],  # theta_0
        [0.6, 1.4],  # theta_1
        [-1.5, 1.5],  # theta_2
        [-1.5, 1.5],  # theta_3
        [0, 1.2],  # theta_4
    ]

    axs.set_xlim(x_limits[element_index])
    axs.set_xticks(np.linspace(x_limits[element_index][0], x_limits[element_index][1], 5))


    sns.kdeplot(        
        true_posterior_samples[:, element_index],
        ax=axs,
        fill=False,
        label="posterior",
        color=truth_color,
        linestyle="-.",
        linewidth=1.5,
        )
    
    sns.kdeplot(
            ps_samples[:, element_index],
            ax=axs,
            fill=False,
            label=f"{model}",
            color=est_color,
            linestyle="-",
            linewidth=1.5,
        )
    
    sns.kdeplot(
            mcmc_samples[:, element_index],
            ax=axs,
            fill=False,
            label=f"{model}+ABC-MCMC",
            color=refined_color,
            linestyle="--",
            linewidth=1.5,
        )
    
    axs.set_title(f"${upper_labels[element_index]}$", pad=15)
    axs.set_ylabel("")

    # save figure
    handles, labels = axs.get_legend_handles_labels()
    fig.legend(handles, labels, loc="lower center", ncol=3)
    plt.tight_layout(pad=3.0)
    plt.savefig(address, dpi=300)
    plt.close(fig)

In [9]:
def compute_mmd(x_,y_, bandwidth):
    """
    Compute the mean and standard deviation of the MMD between x and y.
    :param x_: (N_x,d)
    :param y_: (N_y,d)
    :return:
    """
    x = tf.convert_to_tensor(x_, dtype=tf.float32)
    y = tf.convert_to_tensor(y_, dtype=tf.float32)

    xx = tf.matmul(x,tf.transpose(x))
    xy = tf.matmul(x,tf.transpose(y))
    yy = tf.matmul(y,tf.transpose(y))

    rx = tf.reduce_sum(tf.square(x), axis=-1, keepdims=True)
    ry = tf.reduce_sum(tf.square(y), axis=-1, keepdims=True)

    se_xx = rx - 2*xx + tf.transpose(rx)
    se_xy = rx - 2*xy + tf.transpose(ry)
    se_yy = ry - 2*yy + tf.transpose(ry)

    K_xx = tf.exp(-0.5 * se_xx / (bandwidth**2) )
    K_xy = tf.exp( -0.5 * se_xy / (bandwidth**2) )
    K_yy = tf.exp( -0.5 * se_yy / (bandwidth**2) )

    return tf.reduce_mean(K_xx) - 2*tf.reduce_mean(K_xy) + tf.reduce_mean(K_yy)

## Local ABC-MCMC replicate

In [10]:
nn_replicate_mmd_csv = os.path.join(replicate_result_folder, 'nn_replicate_mmd.csv')
with open(nn_replicate_mmd_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Iteration", "MMD","refined MMD"])

bf_replicate_mmd_csv = os.path.join(replicate_result_folder, 'bf_replicate_mmd.csv')
with open(bf_replicate_mmd_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Iteration", "MMD","refined MMD"])

dnnabc_replicate_mmd_csv = os.path.join(replicate_result_folder, 'dnnabc_replicate_mmd.csv')
with open(dnnabc_replicate_mmd_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Iteration", "MMD","refined MMD"])

w2abc_replicate_mmd_csv = os.path.join(replicate_result_folder, 'w2abc_replicate_mmd.csv')
with open(w2abc_replicate_mmd_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Iteration", "MMD","refined MMD"])

nn_fig_folder = os.path.join(replicate_result_folder, 'nn_figures')
if not os.path.exists(nn_fig_folder):
    os.makedirs(nn_fig_folder)

bf_fig_folder = os.path.join(replicate_result_folder, 'bf_figures')
if not os.path.exists(bf_fig_folder):
    os.makedirs(bf_fig_folder)

In [11]:
burn_in =  250
n_samples = 100
thin = 20
refinement_index = []

for i in range(burn_in+n_samples):
    if i >= burn_in and (i-burn_in) % thin == 0:
        refinement_index.append(i)

print(f"Refinement index: {refinement_index}")

Refinement index: [250, 270, 290, 310, 330]


### update MMD 

In [12]:
for k in range(10):
    print(f"Processing iteration {k+1}...")

    nn_kth_ps_path = os.path.join(nn_ps_samples_path, f'nn_50_ps_{k}.npy')
    nn_ps_samples = np.load(nn_kth_ps_path)

    kth_mcmc_samples_path = os.path.join(nn_mcmc_samples_path, f'nn_50_mcmc_samples_{k}.npy')
    mcmc_samples_total = np.load(kth_mcmc_samples_path)
    mcmc_samples = mcmc_samples_total[refinement_index, :]
    mcmc_samples = mcmc_samples.reshape(-1, d)

    mmd = compute_mmd(nn_ps_samples, true_posterior_samples, bandwidth)
    refined_mmd = compute_mmd(mcmc_samples, true_posterior_samples, bandwidth)

    print(f"MMD: {mmd.numpy()}, Refined MMD: {refined_mmd.numpy()}")
    with open(nn_replicate_mmd_csv, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([k+1, mmd.numpy(), refined_mmd.numpy()])    

Processing iteration 1...
MMD: 0.02628938853740692, Refined MMD: 0.009586632251739502
Processing iteration 2...
MMD: 0.041501596570014954, Refined MMD: 0.012709125876426697
Processing iteration 3...
MMD: 0.032431989908218384, Refined MMD: 0.012495368719100952
Processing iteration 4...
MMD: 0.02409420907497406, Refined MMD: 0.004896253347396851
Processing iteration 5...
MMD: 0.025431714951992035, Refined MMD: 0.010624930262565613
Processing iteration 6...
MMD: 0.0618598610162735, Refined MMD: 0.02592124044895172
Processing iteration 7...
MMD: 0.028445005416870117, Refined MMD: 0.005979865789413452
Processing iteration 8...
MMD: 0.040989942848682404, Refined MMD: 0.024652406573295593
Processing iteration 9...
MMD: 0.02265191078186035, Refined MMD: 0.01273202896118164
Processing iteration 10...
MMD: 0.03902839124202728, Refined MMD: 0.019486211240291595


In [13]:
for k in range(10):
    print(f"Processing iteration {k+1}...")

    bf_kth_ps_path = os.path.join(bf_ps_samples_path, f'bf_50_ps_{k}.npy')
    bf_ps_samples = np.load(bf_kth_ps_path)

    kth_mcmc_samples_path = os.path.join(bf_mcmc_samples_path, f'bf_50_mcmc_samples_{k}.npy')
    mcmc_samples_total = np.load(kth_mcmc_samples_path)
    mcmc_samples = mcmc_samples_total[refinement_index, :]
    mcmc_samples = mcmc_samples.reshape(-1, d)

    mmd = compute_mmd(bf_ps_samples, true_posterior_samples, bandwidth)
    refined_mmd = compute_mmd(mcmc_samples, true_posterior_samples, bandwidth)

    print(f"MMD: {mmd.numpy()}, Refined MMD: {refined_mmd.numpy()}")
    with open(bf_replicate_mmd_csv, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([k+1, mmd.numpy(), refined_mmd.numpy()])

Processing iteration 1...
MMD: 0.007676646113395691, Refined MMD: 0.005471467971801758
Processing iteration 2...
MMD: 0.029050111770629883, Refined MMD: 0.02519504725933075
Processing iteration 3...
MMD: 0.019283801317214966, Refined MMD: 0.009517520666122437
Processing iteration 4...
MMD: 0.01943526417016983, Refined MMD: 0.00873371958732605
Processing iteration 5...
MMD: 0.03170903027057648, Refined MMD: 0.03131464123725891
Processing iteration 6...
MMD: 0.03337589651346207, Refined MMD: 0.020448945462703705
Processing iteration 7...
MMD: 0.028649158775806427, Refined MMD: 0.010161519050598145
Processing iteration 8...
MMD: 0.02474050223827362, Refined MMD: 0.01400686800479889
Processing iteration 9...
MMD: 0.015238858759403229, Refined MMD: 0.01603933423757553
Processing iteration 10...
MMD: 0.03567664325237274, Refined MMD: 0.02978670597076416


In [ ]:
for k in range(10):
    print(f"Processing iteration {k+1}...")

    dnnabc_kth_ps_path = os.path.join(dnnabc_ps_samples_path, f'dnnabc_ps_{k}.npy')
    dnnabc_ps_samples = np.load(dnnabc_kth_ps_path)

    mmd = compute_mmd(dnnabc_ps_samples, true_posterior_samples, bandwidth)
    
    print(f"MMD: {mmd.numpy()}")
    with open(dnnabc_replicate_mmd_csv, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([k+1, mmd.numpy(), "N/A"])

for k in range(10):
    print(f"Processing iteration {k+1}...")

    w2abc_kth_ps_path = os.path.join(w2abc_ps_samples_path, f'w2_ps_{k}.npy')
    w2abc_ps_samples = np.load(w2abc_kth_ps_path)

    mmd = compute_mmd(w2abc_ps_samples, true_posterior_samples, bandwidth)
    
    print(f"MMD: {mmd.numpy()}")
    with open(w2abc_replicate_mmd_csv, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([k+1, mmd.numpy(), "N/A"])

### update Figure

In [71]:
for k in range(10):
    print(f"Processing iteration {k+1}...")

    nn_kth_ps_path = os.path.join(nn_ps_samples_path, f'nn_50_ps_{k}.npy')
    nn_ps_samples = np.load(nn_kth_ps_path)

    kth_mcmc_samples_path = os.path.join(nn_mcmc_samples_path, f'nn_50_mcmc_samples_{k}.npy')
    mcmc_samples_total = np.load(kth_mcmc_samples_path)
    mcmc_samples = mcmc_samples_total[refinement_index, :]
    mcmc_samples = mcmc_samples.reshape(-1, d)

    kth_address = os.path.join(nn_fig_folder, f'nn_50_{k}.png')  
    plot(mcmc_samples, nn_ps_samples, true_posterior_samples, kth_address, model="NN")


Processing iteration 1...
Processing iteration 2...
Processing iteration 3...
Processing iteration 4...
Processing iteration 5...
Processing iteration 6...
Processing iteration 7...
Processing iteration 8...
Processing iteration 9...
Processing iteration 10...


In [72]:
for k in range(10):
    print(f"Processing iteration {k+1}...")

    bf_kth_ps_path = os.path.join(bf_ps_samples_path, f'bf_50_ps_{k}.npy')
    bf_ps_samples = np.load(bf_kth_ps_path)

    kth_mcmc_samples_path = os.path.join(bf_mcmc_samples_path, f'bf_50_mcmc_samples_{k}.npy')
    mcmc_samples_total = np.load(kth_mcmc_samples_path)
    mcmc_samples = mcmc_samples_total[refinement_index, :]
    mcmc_samples = mcmc_samples.reshape(-1, d)

    kth_address = os.path.join(bf_fig_folder, f'bf_50_{k}.png')
    plot(mcmc_samples, bf_ps_samples, true_posterior_samples, kth_address, model="BF")


Processing iteration 1...
Processing iteration 2...
Processing iteration 3...
Processing iteration 4...
Processing iteration 5...
Processing iteration 6...
Processing iteration 7...
Processing iteration 8...
Processing iteration 9...
Processing iteration 10...


### ith element ABC-MCMC trajectory

In [73]:
nn_mcmc_trajectory_folder = os.path.join(replicate_result_folder, 'nn_mcmc_trajectory')
if not os.path.exists(nn_mcmc_trajectory_folder):
    os.makedirs(nn_mcmc_trajectory_folder)
bf_mcmc_trajectory_folder = os.path.join(replicate_result_folder, 'bf_mcmc_trajectory')
if not os.path.exists(bf_mcmc_trajectory_folder):
    os.makedirs(bf_mcmc_trajectory_folder)

In [74]:
for i in range(d):
    print(f"Processing element {i+1}...")
    ith_nn_mcmc_trajectory_subfolder = os.path.join(nn_mcmc_trajectory_folder, f'mcmc_trajectory_{i}')
    if not os.path.exists(ith_nn_mcmc_trajectory_subfolder):
        os.makedirs(ith_nn_mcmc_trajectory_subfolder)

    mcmc_index = []
    for k in range(10):
        print(f"Processing iteration {k+1}...")

        nn_kth_ps_path = os.path.join(nn_ps_samples_path, f'nn_50_ps_{k}.npy')
        nn_ps_samples = np.load(nn_kth_ps_path)
        
        kth_mcmc_samples_path = os.path.join(nn_mcmc_samples_path, f'nn_50_mcmc_samples_{k}.npy')
        mcmc_samples_total = np.load(kth_mcmc_samples_path)

        ik_subfolder = os.path.join(ith_nn_mcmc_trajectory_subfolder, f'{k+1}rounds')
        os.makedirs(ik_subfolder, exist_ok=True)


        for j in range(n_samples+burn_in):
            if (j+1) % thin == 0:
                mcmc_index.append(j)
                if len(mcmc_index) > 5:
                    mcmc_index.pop(0)
                
                mcmc_samples = mcmc_samples_total[mcmc_index, :]
                mcmc_samples = mcmc_samples.reshape(-1, d)
                ikj_address = os.path.join(ik_subfolder, f'{j+1}th_mcmc_samples.png')
                subplot(mcmc_samples, nn_ps_samples, true_posterior_samples, ikj_address, model="NN", element_index=i)


Processing element 1...
Processing iteration 1...
Processing iteration 2...
Processing iteration 3...
Processing iteration 4...
Processing iteration 5...
Processing iteration 6...
Processing iteration 7...
Processing iteration 8...
Processing iteration 9...
Processing iteration 10...
Processing element 2...
Processing iteration 1...
Processing iteration 2...
Processing iteration 3...
Processing iteration 4...
Processing iteration 5...
Processing iteration 6...
Processing iteration 7...
Processing iteration 8...
Processing iteration 9...
Processing iteration 10...
Processing element 3...
Processing iteration 1...
Processing iteration 2...
Processing iteration 3...
Processing iteration 4...
Processing iteration 5...
Processing iteration 6...
Processing iteration 7...
Processing iteration 8...
Processing iteration 9...
Processing iteration 10...
Processing element 4...
Processing iteration 1...
Processing iteration 2...
Processing iteration 3...
Processing iteration 4...
Processing iterat

In [75]:
for i in range(d):
    print(f"Processing element {i+1}...")

    ith_bf_mcmc_trajectory_subfolder = os.path.join(bf_mcmc_trajectory_folder, f'mcmc_trajectory_{i}')
    if not os.path.exists(ith_bf_mcmc_trajectory_subfolder):
        os.makedirs(ith_bf_mcmc_trajectory_subfolder)

    mcmc_index = []
    for k in range(10):
        print(f"Processing iteration {k+1}...")

        bf_kth_ps_path = os.path.join(bf_ps_samples_path, f'bf_50_ps_{k}.npy')
        bf_ps_samples = np.load(bf_kth_ps_path)
        
        kth_mcmc_samples_path = os.path.join(bf_mcmc_samples_path, f'bf_50_mcmc_samples_{k}.npy')
        mcmc_samples_total = np.load(kth_mcmc_samples_path)

        ik_subfolder = os.path.join(ith_bf_mcmc_trajectory_subfolder, f'{k+1}rounds')
        os.makedirs(ik_subfolder, exist_ok=True)


        for j in range(n_samples+burn_in):
            if (j+1) % thin == 0:
                mcmc_index.append(j)
                if len(mcmc_index) > 5:
                    mcmc_index.pop(0)
                
                mcmc_samples = mcmc_samples_total[mcmc_index, :]
                mcmc_samples = mcmc_samples.reshape(-1, d)
                ikj_address = os.path.join(ik_subfolder, f'{j+1}th_mcmc_samples.png')
                subplot(mcmc_samples, bf_ps_samples, true_posterior_samples, ikj_address, model="BF", element_index=i)


Processing element 1...
Processing iteration 1...
Processing iteration 2...
Processing iteration 3...
Processing iteration 4...
Processing iteration 5...
Processing iteration 6...
Processing iteration 7...
Processing iteration 8...
Processing iteration 9...
Processing iteration 10...
Processing element 2...
Processing iteration 1...
Processing iteration 2...
Processing iteration 3...
Processing iteration 4...
Processing iteration 5...
Processing iteration 6...
Processing iteration 7...
Processing iteration 8...
Processing iteration 9...
Processing iteration 10...
Processing element 3...
Processing iteration 1...
Processing iteration 2...
Processing iteration 3...
Processing iteration 4...
Processing iteration 5...
Processing iteration 6...
Processing iteration 7...
Processing iteration 8...
Processing iteration 9...
Processing iteration 10...
Processing element 4...
Processing iteration 1...
Processing iteration 2...
Processing iteration 3...
Processing iteration 4...
Processing iterat